## APLICACIÓN DE MÉTRICAS DE EVALUACIÓN

In [1]:
import os
import sys
os.chdir('..')
sys.path.append(os.getcwd())

In [5]:
from src.app.langraph_core import app
from src.tools.evaluate_rag import LLMJudgeMetric
from src.config import opik_client


def evaluation_task(x: dict) -> dict:
    
    answer = app.invoke({"question": x['Question']})
    context_vec = [answer['documents'][pos].page_content for pos in range(len(answer['documents'])) if answer['documents'][pos] is not None]
    context_vec = '\n'.join(context_vec)
    context_graph = answer['graph_context']
    
    context_general = context_vec + context_graph
    
    return {
        'inputs': x['Question'],
        'outputs': answer['generation'].strip(),
        'context': context_general,
        'reference_outputs': x['Answer'],
        'expected_output': x['Answer'],      
    }

In [ ]:
MODEL_NAME = 'gpt-4o-mini'
correctenes_judge = LLMJudgeMetric(name='Correctness', model_name=MODEL_NAME)
helpfulness_judge = LLMJudgeMetric(name='Helpfulness', model_name=MODEL_NAME)
groundness_judge = LLMJudgeMetric(name='Groundedness', model_name=MODEL_NAME)
retrieval_judge = LLMJudgeMetric(name='Retrieval_relevance', model_name=MODEL_NAME)

metrics = [
    correctenes_judge,
    helpfulness_judge,
    groundness_judge,
    retrieval_judge
]

In [ ]:
dataset = opik_client.get_dataset("usfq-qa-dataset-v2")

eval_results = evaluate(
    experiment_name='large_top-10-gpt-4o-mini-graphRag',
    dataset=dataset,
    task=evaluation_task,
    scoring_metrics=metrics,    
)


In [ ]:
dataset = opik_client.get_dataset("preguntasyrespuestas_clean")

eval_results = evaluate(
    experiment_name='large_top-10-gpt-4o-mini-graphRag',
    dataset=dataset,
    task=evaluation_task,
    scoring_metrics=metrics,    
)

